In [1]:
import argparse
import sys
import time
import os
import prepare_data
import svm
import aspect_term_feature
import aspect_term_polarity_feature
import aspect_category_feature
import aspect_category_polarity_feature
import features
import result
import numpy as np

In [2]:
# we gotta have to purify their training files, since the supplied training files do contain the test sentences as well (this has to be done only once)
'''
dataPath = os.path.join('..','data')    
data.purifyTrainFile(os.path.join(dataPath,'Restaurants_Train.xml'),os.path.join(dataPath,'restaurants-trial.xml'),os.path.join(dataPath,'restaurants_train_purified.xml'))
'''

"\ndataPath = os.path.join('..','data')    \ndata.purifyTrainFile(os.path.join(dataPath,'Restaurants_Train.xml'),os.path.join(dataPath,'restaurants-trial.xml'),os.path.join(dataPath,'restaurants_train_purified.xml'))\n"

In [3]:
start = time.time()
# read training and test data
if prepare_data.useGlove:
    prepare_data.readGloveData()
prepare_data.read()


In [4]:
results = []

for i in range(0,1):
    trainData = prepare_data.train[i]
    testData = prepare_data.test[i]
    print('### Working on domain: ' + prepare_data.domains[i] + ' ###')

    # features that can be used by several feature extractors that we use for different classification tasks
    featuresCommon = features.Features(prepare_data.train[i])
    print('Train and predict aspects ...')
    
    ### classify aspects of a sentence ###
    aspectFeatureExtract = aspect_term_feature.AspectTermFeatures(trainData, featuresCommon)
    sequenceClassifier = svm.SVM(2)
    sequenceClassifier.train(trainData, aspectFeatureExtract)
    sequenceClassifier.predict(testData, aspectFeatureExtract, 'preAspect')

    print('Train and predict aspect sentiments ...')
    ### classify aspect sentiments of a sentence ###
    aspectPolarityFeatureExtract = aspect_term_polarity_feature.AspectTermPolarityFeatures(trainData, featuresCommon)
    multiclassClassifier = svm.SVM(1)
    multiclassClassifier.train(trainData, aspectPolarityFeatureExtract)
    multiclassClassifier.predict(testData, aspectPolarityFeatureExtract, 'preSentAsp')

    # Check if categories are defined for current set
    if len(trainData['categories']) > 0:
        print('Train and predict categories ...')
        for currentCategory in trainData['categories']:
            categoryFeatureExtract = aspect_category_feature.AspectCategoryFeatures(trainData, featuresCommon, currentCategory)
            binaryClassifier = svm.SVM(0)
            binaryClassifier.train(trainData, categoryFeatureExtract)
            binaryClassifier.predict(testData, categoryFeatureExtract, 'preCat')

        print('Train and predict category Sentiments ...')
        for currentCategory in trainData['categories']:
            categoryPolarityFeatureExtractor = aspect_category_polarity_feature.AspectCategoryPolarityFeatures(trainData, featuresCommon, currentCategory)
            multiclassClassifier = svm.SVM(1)
            multiclassClassifier.train(trainData, categoryPolarityFeatureExtractor)
            multiclassClassifier.predict(testData, categoryPolarityFeatureExtractor, 'preCatSent')

    ### evaluate results ###
    results.append(result.evaluate(testData))

end = time.time()
print("Elapsed time: " + str(round(end - start,1)) + "s")
print()

# number to percetage string
def n2P (x): return str(round(100*x,2))

### Working on domain: Restaurant ###
Train and predict aspects ...
Train and predict aspect sentiments ...
Train and predict categories ...
Train and predict category Sentiments ...
Elapsed time: 432.9s



In [6]:
# print final results
print('####################################################################################')
print('Summary of results:')
# print aspect classification results
print('### Aspect Classification ###')
for i in range(len(results)):
    aspectTerm = results[i]['aspects']
    print('# ' + prepare_data.domains[i] + ': ' + n2P(aspectTerm['acc']) + '% \t(precision: ' + n2P(aspectTerm['prec']) + '%, recall: ' + n2P(aspectTerm['rec']) + '%, F1-score: ' + n2P(aspectTerm['f1']) + ')')

####################################################################################
Summary of results:
### Aspect Classification ###
# Restaurant: 96.68% 	(precision: 86.29%, recall: 83.59%, F1-score: 84.92)


In [7]:
# print aspect sentiment classification results
print('### Aspect Sentiment Classification ###')
for i in range(len(results)):
    aspectTermPolarity = results[i]['aspSent']
    print('# ' + prepare_data.domains[i] + ':')
    for j in range(4):
        if aspectTermPolarity[j]['acc'] != -1:
            print('# ' + prepare_data.sents[j] + ': ' + n2P(aspectTermPolarity[j]['acc']) + '% \t(precision: ' + n2P(aspectTermPolarity[j]['prec']) + '%, recall: ' + n2P(aspectTermPolarity[j]['rec']) + '%, F1-score: ' + n2P(aspectTermPolarity[j]['f1']) + ')')
        else:
            print('# ' + prepare_data.sents[j] + ': (Does not appear in test set)')
    print('# -> Average: ' + n2P(aspectTermPolarity[4]['acc']) + '% \t(precision: ' + n2P(aspectTermPolarity[4]['prec']) + '%, recall: ' + n2P(aspectTermPolarity[4]['rec']) + '%, F1-score: ' + n2P(aspectTermPolarity[4]['f1']) + ')')

### Aspect Sentiment Classification ###
# Restaurant:
# Positive: 96.95% 	(precision: 72.66%, recall: 100.0%, F1-score: 84.16)
# Negative: 97.91% 	(precision: 0%, recall: 0.0%, F1-score: 0)
# Conflict: (Does not appear in test set)
# Neutral: 99.04% 	(precision: 0%, recall: 0.0%, F1-score: 0)
# -> Average: 97.96% 	(precision: 24.22%, recall: 33.33%, F1-score: 28.05)


In [8]:
# print category classification results
print('### Category Classification ###')
for i in range(len(results)):
    if 'category' in results[i]:
        category = results[i]['category']
        print('# ' + prepare_data.domains[i] + ':')
        j = 0
        for currentCategory in results[i]['categories']:
            if category[j]['acc'] != -1:
                print('# ' + currentCategory + ': ' + n2P(category[j]['acc']) + '% \t(precision: ' + n2P(category[j]['prec']) + '%, recall: ' + n2P(category[j]['rec']) + '%, F1-score: ' + n2P(category[j]['f1']) + ')')
            else:
                print('# ' + currentCategory + ': (Does not appear in test set)')
            j += 1
        print('# -> Average: ' + n2P(category[j]['acc']) + '% \t(precision: ' + n2P(category[j]['prec']) + '%, recall: ' + n2P(category[j]['rec']) + '%, F1-score: ' + n2P(category[j]['f1']) + ')')

### Category Classification ###
# Restaurant:
# food: 95.0% 	(precision: 97.37%, recall: 90.24%, F1-score: 93.67)
# anecdotes/miscellaneous: 95.0% 	(precision: 95.56%, recall: 93.48%, F1-score: 94.51)
# ambience: 96.0% 	(precision: 100.0%, recall: 42.86%, F1-score: 60.0)
# service: 99.0% 	(precision: 88.89%, recall: 100.0%, F1-score: 94.12)
# price: 99.0% 	(precision: 100.0%, recall: 91.67%, F1-score: 95.65)
# -> Average: 96.8% 	(precision: 96.36%, recall: 83.65%, F1-score: 87.59)


In [11]:
print('### Category Sentiment Classification ###')
for i in range(len(results)):
    if 'category' in results[i]:
        categoryPolarity = results[i]['catSent']
        print('# ' + prepare_data.domains[i] + ':')
        j = 0
        for currentCategory in results[i]['categories']:
            print('#    Category: ' + currentCategory + ':')
            # for each sentiment
            for k in range(4):
                if categoryPolarity[j][k]['acc'] != -1:
                    print('#    ' + prepare_data.sents[k] + ': ' + n2P(categoryPolarity[j][k]['acc']) + '% \t(precision: ' + n2P(categoryPolarity[j][k]['prec']) + '%, recall: ' + n2P(categoryPolarity[j][k]['rec']) + '%, F1-score: ' + n2P(categoryPolarity[j][k]['f1']) + ')')
                else:
                    print('#    ' + prepare_data.sents[k] + ': (Does not appear in test set)')
            print('#    -> Average: ' + n2P(categoryPolarity[j][4]['acc']) + '% \t(precision: ' + n2P(categoryPolarity[j][4]['prec']) + '%, recall: ' + n2P(categoryPolarity[j][4]['rec']) + '%, F1-score: ' + n2P(categoryPolarity[j][4]['f1']) + ')')
            j += 1

        print('# -> Average over all categories:')
        for k in range(4):
            print('# ' + prepare_data.sents[k] + ': ' + n2P(categoryPolarity[j][k]['acc']) + '% \t(precision: ' + n2P(categoryPolarity[j][k]['prec']) + '%, recall: ' + n2P(categoryPolarity[j][k]['rec']) + '%, F1-score: ' + n2P(categoryPolarity[j][k]['f1']) + ')')

print('####################################################################################')

### Category Sentiment Classification ###
# Restaurant:
#    Category: food:
#    Positive: 75.61% 	(precision: 75.61%, recall: 100.0%, F1-score: 86.11)
#    Negative: 78.05% 	(precision: 0%, recall: 0.0%, F1-score: 0)
#    Conflict: 97.56% 	(precision: 0%, recall: 0.0%, F1-score: 0)
#    Neutral: (Does not appear in test set)
#    -> Average: 83.74% 	(precision: 25.2%, recall: 33.33%, F1-score: 28.7)
#    Category: anecdotes/miscellaneous:
#    Positive: 60.87% 	(precision: 88.89%, recall: 50.0%, F1-score: 64.0)
#    Negative: 86.96% 	(precision: 0%, recall: 0.0%, F1-score: 0)
#    Conflict: (Does not appear in test set)
#    Neutral: 56.52% 	(precision: 28.57%, recall: 100.0%, F1-score: 44.44)
#    -> Average: 68.12% 	(precision: 39.15%, recall: 50.0%, F1-score: 36.15)
#    Category: ambience:
#    Positive: 71.43% 	(precision: 71.43%, recall: 100.0%, F1-score: 83.33)
#    Negative: 71.43% 	(precision: 0%, recall: 0.0%, F1-score: 0)
#    Conflict: (Does not appear in test set)
#    N